In [ ]:
import torch
import torchvision
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
#hyper-parameters
epoch=50
batch_size=100
lr=0.1
device=torch.device('cuda:0'if torch.cuda.is_available()else 'cpu')

In [ ]:
%cd /kaggle/input
#Train dataset and Validation dataset
df=pd.read_csv('train.csv')
X=df.drop('label',axis=1).values
y=df['label'].values
#Reshape from x,784 to x,28,28 where X.shape[0] is batch size
X=np.reshape(X,newshape=(X.shape[0],28,28))
#To [0,1]
X=X/255
#Normalization
X=(X-0.5)/0.5
x_train,x_valid,y_train,y_valid=train_test_split(X,y,test_size=0.2,shuffle=True)

x_train,x_valid=torch.from_numpy(x_train).type(torch.FloatTensor),torch.from_numpy(x_valid).type(torch.FloatTensor)
y_train,y_valid=torch.from_numpy(y_train).type(torch.LongTensor),torch.from_numpy(y_valid).type(torch.LongTensor)

#Prepare data for convolutional network
x_train=x_train.unsqueeze_(dim=1)
x_valid=x_valid.unsqueeze_(dim=1)

train=torch.utils.data.TensorDataset(x_train,y_train)
valid=torch.utils.data.TensorDataset(x_valid,y_valid)
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=False)
valid_loader=torch.utils.data.DataLoader(valid,batch_size=batch_size,shuffle=False)


In [ ]:
#Test data
test_df=pd.read_csv('test.csv')
test_data=np.array(test_df,dtype=np.float)


#Reshape
test_data=np.reshape(test_data,newshape=(test_data.shape[0],28,28))

test_data=test_data/255
test_data=(test_data-0.5)/0.5

test_data=torch.from_numpy(test_data).type(torch.FloatTensor)
test_data=test_data.unsqueeze_(dim=1)

test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=False)


In [ ]:
print('Train: ',len(train_loader)*batch_size)
print('Validation: ',len(valid_loader)*batch_size)
print('Test: ',len(test_loader)*batch_size)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2),
                                nn.ReLU(),nn.MaxPool2d(2))#14,14
        self.conv2=nn.Sequential(nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2),
                                nn.ReLU(),nn.MaxPool2d(2))#7,7
        self.linear1=nn.Linear(32*7*7,1024)
        self.linear2=nn.Linear(1024,124)
        self.out=nn.Linear(124,10)
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=x.view(x.size(0),-1)
        x=self.linear1(x)
        x=self.linear2(x)
        output=self.out(x)
        return output

In [ ]:
cnn=CNN()
cnn.cuda()
optimizer=torch.optim.SGD(cnn.parameters(),lr=lr)
loss_func=nn.CrossEntropyLoss().cuda()
print(cnn)

In [ ]:
def Model_Training(epoch,train_loader,valid_loader,loss_func,optimizer,nn):
    train_correct=0
    train_total=0
    valid_correct=0
    valid_total=0
    train_loss=[]
    train_accuracy=[]
    valid_loss=[]
    valid_accuracy=[]
    for epoch in range(epoch):
        print('\nEpoch: {}'.format(epoch))
        print('\nModel training....................................')
        cnn.train()
        for step,(data,labels)in enumerate(train_loader):
            data=data.cuda()
            labels=labels.cuda()
            data=torch.autograd.Variable(data)
            labels=torch.autograd.Variable(labels)
            output=cnn(data)
            loss=loss_func(output,labels)
            predict=torch.max(output.data,1)[1]
            train_correct+=(predict==labels).sum()
            train_total+=labels.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if step%100==0 and step!=0:
                print('Step: ',step,'loss: ',loss.data.cpu().numpy())
        train_loss.append(float(loss.data.cpu().numpy()))
        train_accuracy.append(float(train_correct)/float(train_total))
        print('Model evaluating..................................')
        cnn.eval()
        with torch.no_grad():
            for step,(data,labels)in enumerate(valid_loader):
                data=data.cuda()
                labels=labels.cuda()
                data,labels=torch.autograd.Variable(data),torch.autograd.Variable(labels)
                output=cnn(data)
                loss=loss_func(output,labels)
                predict=torch.max(output.data,1)[1]
                valid_correct+=(predict==labels).sum()
                valid_total+=labels.size(0)



                if step%40==0 and step!=0:
                    print('Step: ',step,'loss: ',loss.data.cpu().numpy())
            valid_loss.append(float(loss.data.cpu().numpy()))
            valid_accuracy.append(float(valid_correct)/float(valid_total))
    print('\nEnd of Training!')
    return train_accuracy,train_loss,valid_accuracy,valid_loss

In [ ]:
train_accuracy,train_loss,valid_accuracy,valid_loss=Model_Training(epoch,train_loader,valid_loader,loss_func,optimizer,cnn)

In [ ]:
print('Train Accuracy: {:.2f}%'.format(train_accuracy[-1]*100))
print('Valid Accuracy: {:.2f}%'.format(valid_accuracy[-1]*100))

In [ ]:
test_label=[]
for image_id,data in enumerate(test_loader):
    data=data.cuda()
    data=torch.autograd.Variable(data)
    output=cnn.forward(data)
    predict=torch.max(output.data,1)[1]
    test_label.append(predict)
    

In [ ]:
test=[]
imageid=list(range(1,28001))
for i in test_label:
    test+=list(i.cpu().numpy())

In [ ]:
dic={'ImageId':imageid,'Label':test}
df_test=pd.DataFrame(dic)
df_test.to_csv('/kaggle/working/submission.csv',index=False)